# Validation
### Checking the relevance of Feature Engineering and correlation to remove unecessary variables and improve the accuracy

The aim is to create a model to predict next headway with:
* all variables
* selected variables

In [57]:
import pandas as pd
import pandas_profiling
import numpy as np
import os
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

ImportError: No module named pandas_profiling

In [2]:
# Choose the city of analysis

city = 'Recife'
# city = 'CG'
# city = 'Curitiba'

In [12]:
path = os.getcwd() + "/../data/output/" + city + "/"

# All variables
all_data = "new_feats_data.csv"
integrated_data_path = path + all_data

# Selected variables
selected_data = "selected_data.csv"
selected_data_path = path + selected_data

In [63]:
# Reading data

df_all_data = pd.read_csv(integrated_data_path)
#df_selected_data = pd.read_csv(selected_data_path)

In [65]:
df_all_data = df_all_data.loc[0:200000]

### Converting NaN values

Changing NaN by median or:
- values of shape for gps

In [47]:
df_all_data.isnull().any()

route                               False
tripNum                             False
shapeId                             False
routeFrequency                      False
shapeSequence                       False
shapeLat                            False
shapeLon                            False
distanceTraveledShape               False
busCode                             False
gpsPointId                           True
gpsLat                               True
gpsLon                               True
distanceToShapePoint                 True
stopPointId                         False
problem                             False
alertSubtype                        False
alertType                           False
alertRoadType                        True
alertConfidence                      True
alertNComments                       True
alertNImages                         True
alertNThumbsUp                       True
alertReliability                     True
alertReportMood                   

In [66]:
# changing by shape data
df_all_data.gpsPointId = np.where(df_all_data.gpsPointId.isnull(), 0, df_all_data.gpsPointId)
df_all_data.gpsLat = np.where(df_all_data.gpsLat.isnull(), df_all_data.shapeLat, df_all_data.gpsLat)
df_all_data.gpsLon = np.where(df_all_data.gpsLon.isnull(), df_all_data.shapeLon, df_all_data.gpsLon)
df_all_data.distanceToShapePoint = np.where(df_all_data.distanceToShapePoint.isnull(), 
                                            0, df_all_data.distanceToShapePoint)


In [67]:
#grouping by weekday and filling NA's with the median of the column
numeric_feats = df_all_data.dtypes[(df_all_data.dtypes != "object") & (df_all_data.dtypes != "datetime64[ns]")
                               & (df_all_data.dtypes != "bool")].index
df_all_data[numeric_feats] = df_all_data.groupby(['WEEKDAY(gps_datetime)'])[numeric_feats].transform(lambda x: x.fillna(x.median()))

df_all_data.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,DAY(jamUpdateDateTime),YEAR(gps_datetime),YEAR(alertDateTime),YEAR(jamUpdateDateTime),MONTH(gps_datetime),MONTH(alertDateTime),MONTH(jamUpdateDateTime),WEEKDAY(gps_datetime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime)
0,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50665,0.0,...,1.0,2018,2018.0,2018.0,12,12.0,12.0,5,5.0,5.0
1,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50673,0.0,...,1.0,2018,2018.0,2018.0,12,12.0,12.0,5,5.0,5.0
2,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50662,0.0,...,1.0,2018,2018.0,2018.0,12,12.0,12.0,5,5.0,5.0
3,96,2,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50673,0.0,...,1.0,2018,2018.0,2018.0,12,12.0,12.0,5,5.0,5.0
4,96,2,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50662,176265.0,...,1.0,2018,2018.0,2018.0,12,12.0,12.0,5,5.0,5.0


In [50]:
df_all_data.isnull().any()

route                               False
tripNum                             False
shapeId                             False
routeFrequency                      False
shapeSequence                       False
shapeLat                            False
shapeLon                            False
distanceTraveledShape               False
busCode                             False
gpsPointId                          False
gpsLat                              False
gpsLon                              False
distanceToShapePoint                False
stopPointId                         False
problem                             False
alertSubtype                        False
alertType                           False
alertRoadType                       False
alertConfidence                     False
alertNComments                      False
alertNImages                        False
alertNThumbsUp                      False
alertReliability                    False
alertReportMood                   

In [55]:
df_all_data.dtypes

route                                                int64
tripNum                                              int64
shapeSequence                                        int64
shapeLat                                           float64
shapeLon                                           float64
distanceTraveledShape                              float64
busCode                                              int64
gpsPointId                                         float64
gpsLat                                             float64
gpsLon                                             float64
distanceToShapePoint                               float64
stopPointId                                          int64
alertRoadType                                      float64
alertConfidence                                    float64
alertNComments                                     float64
alertNImages                                       float64
alertNThumbsUp                                     float

### Converting categorical variables

In [68]:
df_all_data = pd.get_dummies(df_all_data)
df_all_data.head()

,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertSubtype_ROAD_CLOSED_EVENT,alertType_-,alertType_ACCIDENT,alertType_CHIT_CHAT,alertType_HAZARD,alertType_JAM,alertType_POLICE,alertType_ROAD_CLOSED,jamBlockType_-,jamBlockType_ROAD_CLOSED_EVENT
0,96,1,22857,-8.046589,-34.878125,3606.0,50665,0.0,-8.046589,-34.878125,...,0,0,1,0,0,0,0,0,1,0
1,96,1,22857,-8.046589,-34.878125,3606.0,50673,0.0,-8.046589,-34.878125,...,0,0,1,0,0,0,0,0,1,0
2,96,1,22857,-8.046589,-34.878125,3606.0,50662,0.0,-8.046589,-34.878125,...,0,0,1,0,0,0,0,0,1,0
3,96,2,22857,-8.046589,-34.878125,3606.0,50673,0.0,-8.046589,-34.878125,...,0,0,1,0,0,0,0,0,1,0
4,96,2,22857,-8.046589,-34.878125,3606.0,50662,176265.0,-8.046504,-34.878170,...,0,0,1,0,0,0,0,0,1,0


In [58]:
# Descriptive statistics
profile = pandas_profiling.ProfileReport(df_all_data, check_correlation=True, correlation_threshold=0.5)

NameError: name 'pandas_profiling' is not defined

### Training prediction model

In [69]:
y_train_all = df_all_data.drop('busBunching', axis=1)
y_train_all = y_train_all.astype('bool')
X_train_all = df_all_data

#### Modelo RF (Random Forest)
Random Forest é um algoritmo que ajusta várias árvores de decisão de classificação em várias subamostras do conjunto de dados e usa a média para melhorar a precisão preditiva e controlar o ajuste excessivo. A quantidade de árvores (n_estimators) é um parâmetro, cujo valor, deve ser encontrado, para que o modelo melhor represente os dados.

In [70]:
# function to train the model and to calculate the RMSE with cross validation
def rmse_cv(model, X_train, y_train):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 10))
    return(rmse)

# function to print the coefficients of the model
def print_coefficients(model):
    w = list(model.coef_)
    # Parameters must be in reverse order because the poly function 
    w.reverse()
    print (np.poly1d(w) + model.intercept_)

# function to train the model and to return the score
def train_model(model, X_train, y_train, print_coef):
    model.fit(X_train, y_train)
    if (print_coef):
        print_coefficients(model)
    
    score = model.score(X_train, y_train)
    return score

# function to plot the RMSE vs parameter value
def plot_rmse_param(series, param_name):
    series.plot(title = "Validation Error vs " + param_name)
    plt.xlabel(param_name)
    plt.ylabel("RMSE")
    
# function to get the best RMSE and the best parameter value of the model
def best_rmse_param(series):
    best_rmse = series.min()
    best_param = series.idxmin() 
    
    return(best_rmse, best_param)

In [ ]:
n_estimators = [1, 10, 50, 100]
cv_rf_rmse = [rmse_cv(RandomForestClassifier(n_estimators = n), X_train_all, y_train_all).mean() 
            for n in n_estimators]

series = pd.Series(cv_rf_rmse, index = n_estimators)
plot_rmse_param(series, "n_estimators")

In [ ]:
best_rmse_rf, best_param_rf = best_rmse_param(series)

model_rf = RandomForestClassifier(n_estimators = best_param_rf)
score_rf = train_model(model_rf, X_train_all, y_train_all, False)

print u"Modelo Random Forest: \nEstimators = {0} \nRMSE = {1} \nScore = {2}".format(best_param_rf, best_rmse_rf, score_rf)